In [8]:
# -*- coding: utf-8 -*-
import pandas as pd
import re
import os

# 파일 경로 설정
input_path = r"C:\Users\user\Downloads\바종설-슈는시간(응답) - 설문지 응답 시트1.csv"
output_path = r"C:\Users\user\Downloads\결과_계산.csv"

# CSV 읽기
df = pd.read_csv(input_path, encoding='cp949')


# 열 이름 매핑 (가독성을 위해 변수로)
expected_time_col = '여러분이 예상하는 핸드폰 사용시간은 얼마인가요?\n(ex. 2시간 22분)'
expected_count_col = '오늘 하루 동안 본인의 휴대폰 화면을 몇 회 정도 켰다고 생각하시나요? \n(ex. 22회)'
actual_time_col = '실제 본인의 스크린타임을 알려주세요!\n(ex. 10시간 52분)'
actual_count_col = '실제 본인의 휴대폰 확인 횟수를 알려주세요!\n(ex. 137회)'
time_compare_col = '실제 휴대폰 사용 시간이 예상보다 많았나요?'
count_compare_col = '실제 휴대폰 확인량이 예상보다 많았나요?'

# "n시간 n분" → 소수점 시간
def time_to_hours(time_str):
    if pd.isna(time_str):
        return 0.0
    hours = 0
    minutes = 0
    h_match = re.search(r'(\d+)\s*시간', time_str)
    m_match = re.search(r'(\d+)\s*분', time_str)
    if h_match:
        hours = int(h_match.group(1))
    if m_match:
        minutes = int(m_match.group(1))
    return hours + minutes / 60

# "n회" → 정수 추출
def count_from_text(count_str):
    if pd.isna(count_str):
        return 0
    match = re.search(r'(\d+)', str(count_str))
    return int(match.group(1)) if match else 0

# 결과 계산
time_diffs = []
count_diffs = []

for index, row in df.iterrows():
    time_diff = None
    count_diff = None

    if str(row.get(time_compare_col)).strip() == '네':
        start = time_to_hours(row.get(expected_time_col))
        end = time_to_hours(row.get(actual_time_col))
        time_diff = end - start

    if str(row.get(count_compare_col)).strip() == '네':
        count_start = count_from_text(row.get(expected_count_col))
        count_end = count_from_text(row.get(actual_count_col))
        count_diff = count_end - count_start

    time_diffs.append(time_diff)
    count_diffs.append(count_diff)

# 결과 데이터프레임 생성
result_df = pd.DataFrame({
    "번호": range(1, len(df) + 1),
    "시간 연산 결과": time_diffs,
    "횟수 연산 결과": count_diffs
})

# 유효한 값 기준으로 통계 계산
valid_time_diffs = [x for x in time_diffs if x is not None]
valid_count_diffs = [x for x in count_diffs if x is not None]

sum_row = {
    "번호": "총합",
    "시간 연산 결과": sum(valid_time_diffs) if valid_time_diffs else None,
    "횟수 연산 결과": sum(valid_count_diffs) if valid_count_diffs else None
}
avg_row = {
    "번호": "평균",
    "시간 연산 결과": (sum(valid_time_diffs) / len(valid_time_diffs)) if valid_time_diffs else None,
    "횟수 연산 결과": (sum(valid_count_diffs) / len(valid_count_diffs)) if valid_count_diffs else None
}

# 통계 추가
result_df = pd.concat([result_df, pd.DataFrame([sum_row, avg_row])], ignore_index=True)

# 저장
result_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"결과가 저장되었습니다: {output_path}")


결과가 저장되었습니다: C:\Users\user\Downloads\결과_계산.csv
